In [56]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.chat_history import BaseChatMessageHistory
from langchain.schema.runnable.history import RunnableWithMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

import os
import json

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [57]:
transcribations = []
for i in range(1, 7):
    with open(f'./data/audio{i}.json', 'r') as f:
        transcribations.append(json.load(f))

In [61]:
transcribation = transcribations[0]

full_text = '\n'.join([seg['text'].strip() for seg in transcribation['segments']])
full_text = full_text.replace('  ', ' ').strip()

In [62]:
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="IlyaGusev/saiga_mistral_7b_merged",
    task="text-generation",
    device=0,  # -1 for CPU
    batch_size=2,  # adjust as needed based on GPU map and model size.
    model_kwargs={
        "temperature": 0.01, 
        "max_length": 1536, 
        "exponential_decay_length_penalty": (1280, 2.5),
        "top_p": 0.9, 
        "do_sample": True},
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.43s/it]


## Exctract intro and conclusion

In [63]:
EXTRACT_INTRO_PROMPT = '''
Тебе будет дан текст, сожержащий начало лекции. Напиши вступление для конспекта этой лекции.
Вступление обязательно содержит 3 пункта: 
1. тема лекции
2. какие вопросы будут рассматриваться в лекции
3. обосновывается актуальность и значимость темы (зачем студенту изучать эту лекцию, каких ошибок студент избежит освоив эту тему).

Текст:
BEGIN_TEXT
%s
END_TEXT
Ответ:
'''

In [64]:
EXTRACT_CONCLUSION_PROMPT = '''
Тебе будет дан текст, сожержащий конец лекции. Напиши заключение для конспекта этой лекции.
В заключении должно быть два-три предложения, подводящих итог лекции, кратко описывающих пройденный материал и знания, полученные на лекции.

Текст:
BEGIN_TEXT
%s
END_TEXT
Ответ:
'''

In [65]:
# DELETE_INTRO_PROMPT = '''
# Тебе будет дан текст, сожержащий начало лекции. Удали из него вступительное слово.
# Во вступлении может обсуждаться тема лекции, какие вопросы будут рассматриваться в лекции, а также актуальность.
# Удали из текста все предложения, относящиеся к вступлению и оставь только основную часть.

# Текст:
# BEGIN_TEXT
# %s
# END_TEXT
# Ответ:
# '''

In [66]:
# DELETE_CONCLUSION_PROMPT = '''
# Тебе будет дан текст, сожержащий конец лекции. Удали из него заключение.
# Заключение может содержать несколько предложения, подводящих итог лекции, кратко описывающих пройденный материал и знания, полученные на лекции.
# Удали из текста все предложения, относящиеся к заключению и оставь только основную часть.

# Текст:
# BEGIN_TEXT
# %s
# END_TEXT
# Ответ:
# '''

In [67]:
BASE_PROMPT = '''
{prompt}
'''

In [68]:
tokenizer = AutoTokenizer.from_pretrained('IlyaGusev/saiga_mistral_7b_merged')
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size = 512,  # 768
    chunk_overlap = 128,
    add_start_index = True,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [69]:
chunks = text_splitter.create_documents([full_text])
chunks = [item.page_content for item in chunks]

In [70]:
definition_retrieval_system = """Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.""".strip()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", definition_retrieval_system),
        ("human", BASE_PROMPT)
    ]
)

gpu_chain = prompt | gpu_llm.bind(stop=[" bot"]) 

In [71]:
questions = [{'prompt': cur_text} for cur_text in (EXTRACT_INTRO_PROMPT % chunks[0], EXTRACT_CONCLUSION_PROMPT % chunks[-1])]
intro, conclusion = gpu_chain.batch(questions)

/data/vkarlov/venvs/langchain/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


In [72]:
print(intro)

Тема лекции: Массивы в языке программирования C-Sharp

Вопросы, которые будут рассматриваться в лекции:
1. Основные понятия, связанные с массивами
2. Применение массивов в различных задачах
3. Операции создания, заполнения и вывода массивов на экран
4. Разновидности циклов FOR и FOREACH
5. Изучение английского для программистов

Актуальность и значимость темы:
1. Массивы являются одним из основных типов структур данных в программировании, которые используются для хранения и обработки больших объемов информации.
2. Они позволяют эффективно решать задачи, связанные с обработкой данных, такие как поиск, сортировка, фильтрация и т.д.
3. Изучение массивов в рамках языка программирования C-Sharp позволяет студентам освоить основные навыки работы с данными и применить их в решении различных задач.
4. Развитие навыков работы с массивами помогает студентам избежать ошибок при работе с большими объемами данных и улучшить эффективность программного кода.
5. Изучение английского языка для программ

In [73]:
print(conclusion)

В заключении лекции можно подвести, что мы рассмотрели основные характеристики массивов, изучили базовые операции с ними, а также изучили разновидности циклов FOR и FOREACH. Кроме того, мы обсудили важность изучения технического английского и его применения в программировании.


## Split and get themes

In [74]:
transcribation = transcribations[0]

full_text = ' '.join([seg['text'].strip() for seg in transcribation['segments']])
full_text = full_text.replace('  ', ' ').strip()

In [75]:
import numpy as np
import spacy

# Load the Spacy model
nlp = spacy.load('ru_core_news_sm')

def process(text):
    doc = nlp(text)
    sents = list(doc.sents)
    vecs = np.stack([sent.vector / sent.vector_norm for sent in sents])

    return sents, vecs

def cluster_text(sents, vecs, threshold):
    clusters = [[0]]
    for i in range(1, len(sents)):
        if np.dot(vecs[i], vecs[i-1]) < threshold:
            clusters.append([])
        clusters[-1].append(i)
    
    return clusters

def clean_text(text):
    # Add your text cleaning process here
    return text

In [76]:
def split_into_paragraphs(input_text):
    # Initialize the clusters lengths list and final texts list
    clusters_lens = []
    final_texts = []

    # Process the chunk
    threshold = 0.05
    sents, vecs = process(input_text)

    # Cluster the sentences
    clusters = cluster_text(sents, vecs, threshold)

    for cluster in clusters:
        cluster_txt = clean_text(' '.join([sents[i].text for i in cluster]))
        cluster_len = len(cluster_txt)
        
        # Check if the cluster is too short
        if cluster_len < 800:
            continue
        
        # Check if the cluster is too long
        elif cluster_len > 3000:
            threshold = 0.1
            sents_div, vecs_div = process(cluster_txt)
            reclusters = cluster_text(sents_div, vecs_div, threshold)
            
            for subcluster in reclusters:
                div_txt = clean_text(' '.join([sents_div[i].text for i in subcluster]))
                div_len = len(div_txt)
                
                if div_len < 800 or div_len > 3000:
                    continue
                
                clusters_lens.append(div_len)
                final_texts.append(div_txt)
                
        else:
            clusters_lens.append(cluster_len)
            final_texts.append(cluster_txt)
            
    return final_texts

In [77]:
paragraphs = split_into_paragraphs(full_text)

In [78]:
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="IlyaGusev/saiga_mistral_7b_merged",
    task="text-generation",
    device=0,  # -1 for CPU
    batch_size=16,  # adjust as needed based on GPU map and model size.
    model_kwargs={
        "temperature": 0.01,
        "max_length": 2048,
        # "exponential_decay_length_penalty": (2000, 2.5),
        "top_p": 0.9, 
        "do_sample": True},
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.73s/it]


In [79]:
THEME_PROMPT = '''
Тебе будет дан текст, сожержащий отрывок лекции. Придумай короткую тему для данного отрывка, отражающую его суть.
Придуманная тема не должна быть длинной, она должна содержать меньше 8 слов.

Текст:
BEGIN_TEXT
{text}
END_TEXT
Ответ:
'''

In [80]:
definition_retrieval_system = """Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.""".strip()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", definition_retrieval_system),
        ("human", THEME_PROMPT)
    ]
)

gpu_chain = prompt | gpu_llm.bind(stop=[" bot"]) 

In [81]:
questions = [{'text': cur_text} for cur_text in paragraphs]
tokenizer_kwargs={
    'truncation':True,'max_length':1536
}
themes = gpu_chain.batch(questions, tokenizer_kwargs)

/data/vkarlov/venvs/langchain/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


In [82]:
i = 4
print(themes[i])
print('=' * 100)
print(paragraphs[i])

Четные числа в массиве
Двигаемся далее. И давайте рассмотрим типовую задачу на поиске элементов в массиве, которые удовлетворяют некоторым условиям. Пусть у нас имеется массив на некоторое количество целых чисел, допустим, на 10 штук. Требуется найти в нем четные числа и вывести их на экран. Друзья, напомню, что число является четным, если оно делится нацело на 2. Примеры четных чисел это 0, 2, 4, 6 и так далее. Давайте для этой задачи перечислим основные этапы, сформируем ее блок-схему и реализуем эту схему на языке C-sharp. Итак, в этой задаче можно выделить следующие этапы. Первое – создать и заполнить массив на 10 цветочных чисел. Вторым этапом можно выделить проход по элементам, то есть просмотр каждого элемента. Третий этап – это проверка элемента на условия того, является ли он четным. Ну и четвертый этап, друзья, это вывести этот элемент на экран, если указанное условие выполняется.


In [ ]:
import joblib
joblib.dump(themes, 'themes.joblib')
joblib.dump(paragraphs, 'paragraphs.joblib')

['paragraphs.joblib']

## Construct document

In [83]:
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
import re

In [84]:
def format_docx(introduction, main_body, conclusion, doc_name='abstract'):
    document = Document()
    document.add_heading('Конспект лекции', 0)
    document.add_heading(f"Введение", level=1)

    SENTENCE1 = 'Тема лекции'
    SENTENCE2 = 'Вопросы, которые будут рассматриваться в лекции'
    SENTENCE3 = 'Актуальность и значимость темы'

    ind_start1 = introduction.find(SENTENCE1) 
    if ind_start1 == -1:
        document.add_paragraph(introduction)
        document.add_heading(f"Основная часть", level=1)
        document.add_paragraph(main_body)

        document.add_heading(f"Заключение", level=1)
        document.add_paragraph(conclusion)
        document.save(f'{doc_name}.docx')
        return document
    
    introduction = introduction[ind_start1:]
    document.add_heading(f"{SENTENCE1}:", level=3)
    introduction = re.sub(f"{SENTENCE1}:", '', introduction)

    ind_start2 = introduction.find(SENTENCE2)
    if ind_start2 == -1:
        document = Document()
        document.add_heading('Конспект лекции', 0)
        document.add_heading(f"Введение", level=1)
        document.add_paragraph(introduction)
        document.add_heading(f"Основная часть", level=1)
        document.add_paragraph(main_body)

        document.add_heading(f"Заключение", level=1)
        document.add_paragraph(conclusion)
        document.save(f'{doc_name}.docx')
        return document
    document.add_paragraph(introduction[:ind_start2].strip())

    introduction = introduction[ind_start2:]
    introduction = re.sub(f"{SENTENCE2}:", '', introduction)

    document.add_heading(f"{SENTENCE2}:", level=3)
    ind_start3 = introduction.find(SENTENCE3)
    if ind_start2 == -1:
        document = Document()
        document.add_heading('Конспект лекции', 0)
        document.add_heading(f"Введение", level=1)
        document.add_paragraph(introduction)
        document.add_heading(f"Основная часть", level=1)
        document.add_paragraph(main_body)

        document.add_heading(f"Заключение", level=1)
        document.add_paragraph(conclusion)
        document.save(f'{doc_name}.docx')
        return document
    
    document.add_paragraph(introduction[: ind_start3].strip())

    introduction = introduction[ind_start3:]
    introduction = re.sub(f"{SENTENCE3}:", '', introduction)
    document.add_heading(f"{SENTENCE3}:", level=3)
    
    document.add_paragraph(introduction[:].strip())
    # document.add_page_break()

    # Обработка основной части
    themes, paragraphs = main_body
    document.add_heading(f"Основная часть", level=1)
    for theme, paragraph in zip(themes, paragraphs):    
        document.add_heading(theme, level=3)
        document.add_paragraph('\t' + paragraph)

    document.add_heading(f"Заключение", level=1)
    document.add_paragraph('\t' + conclusion.strip())

    document.save(f'{doc_name}.docx')

    return document


In [85]:
format_docx(intro, (themes, paragraphs), conclusion)

## Constuct front

In [86]:
def format_front(introduction, main_body, conclusion):
    no_intro_format_flag = 0

    # Introduction
    intro = {
        "title": f"Введение",
        "subtitles": []
    }
    
    SENTENCE1 = 'Тема лекции'
    SENTENCE2 = 'Вопросы, которые будут рассматриваться в лекции'
    SENTENCE3 = 'Актуальность и значимость темы'

    ## тема
    ind_start1 = introduction.find(SENTENCE1) 
    if ind_start1 == -1:
        no_intro_format_flag = 1
    introduction = introduction[ind_start1:]
    introduction = re.sub(f"{SENTENCE1}:", '', introduction)
    ind_start2 = introduction.find(SENTENCE2)
    if ind_start2 == -1:
        no_intro_format_flag = 1
    intro['subtitles'].append(
            {
                "subtitle": f"{SENTENCE1}:",
                "texts": [{ 'text': introduction[:ind_start2].strip()}]
            }
    )
    ## вопросы
    introduction = introduction[ind_start2:]
    introduction = re.sub(f"{SENTENCE2}:", '', introduction)
    ind_start3 = introduction.find(SENTENCE3)
    if ind_start2 == -1:
        no_intro_format_flag = 1
    intro['subtitles'].append(
            {
                "subtitle": f"{SENTENCE2}:",
                "texts": [{ 'text': introduction[: ind_start3].strip()}]
            }
    )
    ## актуальность
    introduction = introduction[ind_start3:]
    introduction = re.sub(f"{SENTENCE3}:", '', introduction)
    intro['subtitles'].append(
            {
                "subtitle": f"{SENTENCE3}:",
                "texts": [{ 'text': introduction[:].strip()}]
            }
    )
    ## cобираем вместе, если что-то не распарсилось, то сбрасываем форматирование
    if no_intro_format_flag:
        intro['subtitles'] = [
            {
                "subtitle": None,
                "texts": [{ 'text': introduction }]
            }
        ]
        
    # Обработка основной части
    themes, paragraphs = main_body
    main = {
        "title": f"Основная часть",
        "subtitles": []
    }
    for theme, paragraph in zip(themes, paragraphs): 
        main['subtitles'].append(
            {
                "subtitle": theme,
                "texts": [{ 'text': '\t' + paragraph }]
            }
        )

    # Обработка заключения
    concl = {
        "title": f"Заключение",
        "subtitles": []
    }
    concl['subtitles'].append(
        {
            "subtitle": None,
            "texts": ['\t' + conclusion.strip()]
        }
    )

    return [intro, main, concl]


In [89]:
front_output = format_front(intro, (themes, paragraphs), conclusion)

In [92]:
with open('best_front.txt', 'w') as f:
    print(front_output, file=f)

In [91]:
with open('best_front.json', 'w') as f:
    json.dump(front_output, f)

## Extract timecodes from whisper

In [7]:
transcribation = transcribations[0]

In [8]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.chat_history import BaseChatMessageHistory
from langchain.schema.runnable.history import RunnableWithMessageHistory
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [95]:
from langchain_core.documents.base import Document

In [112]:
transcribation['segments'][0]['start']

5.336

In [114]:
asr_texts_source = [Document(page_content=cur_segment['text'], metadata={'start': cur_segment['start']}) for cur_segment in transcribation['segments']]

In [115]:
asr_texts_source[0]

Document(page_content=' Друзья, привет!', metadata={'start': 5.336})

In [98]:
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# asr_texts_source = [cur_segment['text'] for cur_segment in transcribation['segments']]
# asr_text_db = Chroma.from_texts(asr_texts_source, embedding=embedder)
asr_text_db = Chroma.from_documents(asr_texts_source, embedding=embedder)
asr_text_retriever = asr_text_db.as_retriever()

In [108]:
timecodes = {}  # термин -- старт

all_terms = ['массив', 'число', 'рекурсия', 'полет']
for term in all_terms:
    top_doc = asr_text_retriever.invoke(f'определение {term}')[0]
    print(top_doc.page_content, top_doc.metadata)
    # for i, segment in enumerate(transcribation['segments']):
    #     if segment['text'].find(top_doc) != -1:
    #         print()
    #         timecodes[term] = segment['start']

Начнем с ключевого термина этой лекции – массив. {}
В данном случае это целое число. {}
Тут некоторые значения по умолчанию нам {}
Тут некоторые значения по умолчанию нам {}


In [100]:
timecodes

{'массив': 66.493, 'число': 3159.633}

In [48]:
term = 'полет'
asr_text_retriever.invoke(f'определение {term}')

[Document(page_content='Тут некоторые значения по умолчанию нам'),
 Document(page_content=' который мы выделили для массива.'),
 Document(page_content='В, например, намокоз'),
 Document(page_content='Четных, а также добавим нечетные.')]

In [46]:
for i, segment in enumerate(transcribation['segments']):
    if segment['text'] == 'Тут некоторые значения по умолчанию нам': print(segment['text']) 

In [34]:
timecodes

{'массив': 66.493, 'число': 3159.633}

In [29]:
top_doc = asr_text_retriever.invoke('определение массив')[0].page_content

In [17]:
transcribation['segments'][0]['start']

5.336

In [19]:
timecodes = []
for i, segment in enumerate(transcribation['segments']):
    if segment['text'] == top_doc:
        print(segment['start'])

66.493
